## Step 1 - load lib, setup logging

In [1]:
import logging
import coloredlogs
import asyncio 

from ph4_walkingpad import pad
from ph4_walkingpad.pad import Scanner, WalkingPad, WalkingPadCurStatus, WalkingPadLastStatus, Controller
from ph4_walkingpad.utils import setup_logging

log = setup_logging()
pad.logger = log
#coloredlogs.install(level=logging.INFO)

## Step 2 - scan devices

In [2]:
scanner = Scanner()
await scanner.scan() #If the walkingpad is named 'walkingpad'
#await scanner.scan(customname='re') #Set a custom name for scanning. My pad is named 'RE'


Scanning for peripherals...
Device: [ 0], 5E00000D-5512-4000-825B-731000000000, WalkingPad, ['0000fe00-0000-1000-8000-00805f9b34fb']


## Step 3 - connect to the WalkingPad

In [3]:
ctler = Controller()
await ctler.run('5E00000D-5512-4000-825B-731000000000')  # set UUID from the scanning (OSX), or MAC address

# Note that OSX 12+ does not allow using address directly, use scanner result instead
await ctler.run(scanner.walking_belt_candidates[0])

Connected: True
[Service] 0000180a-0000-1000-8000-00805f9b34fb: Device Information
	[Characteristic] 00002a25-0000-1000-8000-00805f9b34fb: (Handle: 9) (read) | Name: Serial Number String, Value: None 
	[Characteristic] 00002a26-0000-1000-8000-00805f9b34fb: (Handle: 11) (read) | Name: Firmware Revision String, Value: None 
	[Characteristic] 00002a28-0000-1000-8000-00805f9b34fb: (Handle: 13) (read) | Name: Software Revision String, Value: None 
	[Characteristic] 00002a24-0000-1000-8000-00805f9b34fb: (Handle: 15) (read) | Name: Model Number String, Value: None 
	[Characteristic] 00002a29-0000-1000-8000-00805f9b34fb: (Handle: 17) (read) | Name: Manufacturer Name String, Value: None 
[Service] 0000fe00-0000-1000-8000-00805f9b34fb: Vendor specific
	[Characteristic] 0000fe01-0000-1000-8000-00805f9b34fb: (Handle: 20) (read,notify) | Name: Vendor specific, Value: None 
		[Descriptor] 00002902-0000-1000-8000-00805f9b34fb: (Handle: 22) | Value: b'\x00\x00' 
	[Characteristic] 0000fe02-0000-1000-80

## Start belt

In [4]:
await ctler.switch_mode(WalkingPad.MODE_MANUAL)
await asyncio.sleep(1.0)
await ctler.start_belt()

Msg: f8, a2, 00, 00, 01, 00, 00, 00, 00, 00, 00, 00, 00, 00, 01, 00, 81, 00, 25, fd
Status: WalkingPadCurStatus(dist=0.0, time=0, steps=0, speed=0.0, state=0, mode=1, app_speed=0.03333333333333333, button=1, rest=0000)
Msg: f8, a2, 09, 00, 01, 00, 00, 00, 00, 00, 00, 00, 00, 00, 01, 00, 03, 00, b0, fd
Status: WalkingPadCurStatus(dist=0.0, time=0, steps=0, speed=0.0, state=9, mode=1, app_speed=0.03333333333333333, button=1, rest=0000)


## Fetch statistics

In [5]:
for i in range(5):
    await ctler.ask_stats()
    await asyncio.sleep(1.0)

Msg: f8, a2, 01, 14, 01, 00, 00, 08, 00, 00, 00, 00, 00, 05, 01, 00, 00, 00, c6, fd
Status: WalkingPadCurStatus(dist=0.0, time=8, steps=5, speed=2.0, state=1, mode=1, app_speed=0.03333333333333333, button=1, rest=0000)
Msg: f8, a2, 01, 14, 01, 00, 00, 09, 00, 00, 00, 00, 00, 06, 01, 00, 00, 00, c8, fd
Status: WalkingPadCurStatus(dist=0.0, time=9, steps=6, speed=2.0, state=1, mode=1, app_speed=0.03333333333333333, button=1, rest=0000)
Msg: f8, a2, 01, 14, 01, 00, 00, 0a, 00, 00, 00, 00, 00, 07, 01, 00, 00, 00, ca, fd
Status: WalkingPadCurStatus(dist=0.0, time=10, steps=7, speed=2.0, state=1, mode=1, app_speed=0.03333333333333333, button=1, rest=0000)
Msg: f8, a2, 01, 14, 01, 00, 00, 0b, 00, 00, 00, 00, 00, 09, 01, 00, 00, 00, cd, fd
Status: WalkingPadCurStatus(dist=0.0, time=11, steps=9, speed=2.0, state=1, mode=1, app_speed=0.03333333333333333, button=1, rest=0000)
Msg: f8, a2, 01, 14, 01, 00, 00, 0c, 00, 00, 00, 00, 00, 0a, 01, 00, 00, 00, cf, fd
Status: WalkingPadCurStatus(dist=0.0, 

## Speed change


In [6]:
await ctler.change_speed(18)

Msg: f8, a2, 01, 14, 01, 00, 00, 15, 00, 00, 01, 00, 00, 16, 36, 00, 00, 00, 1a, fd
Status: WalkingPadCurStatus(dist=0.01, time=21, steps=22, speed=2.0, state=1, mode=1, app_speed=1.8, button=1, rest=0000)


## Ask history record

In [7]:
await ctler.ask_hist(0)

Msg: f8, a7, 00, 1c, 2d, 00, 11, 97, 00, 00, 8c, 00, 00, 05, 00, 00, b9, 01, e3, fd
Record: WalkingPadLastStatus(dist=0.05, time=140, steps=185, rest=001c2d001197)


## Stop belt

In [12]:
# Stop belt
await ctler.stop_belt()
await asyncio.sleep(1.0)

# Standby
await ctler.switch_mode(WalkingPad.MODE_STANDBY)

Msg: f8, a2, 01, 12, 01, 00, 00, 40, 00, 00, 03, 00, 00, 46, 00, 00, 00, 00, 3f, fd
Status: WalkingPadCurStatus(dist=0.03, time=64, steps=70, speed=1.8, state=1, mode=1, app_speed=0, button=1, rest=0000)
Msg: f8, a2, 01, 09, 02, 00, 00, 41, 00, 00, 03, 00, 00, 48, 02, 00, 00, 00, 3c, fd
Status: WalkingPadCurStatus(dist=0.03, time=65, steps=72, speed=0.9, state=1, mode=2, app_speed=0.06666666666666667, button=2, rest=0000)


## Disconnect Bluetooth LE

In [13]:
# Call before terminating
await ctler.disconnect()